# Building the networks

## Initial steps

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

path = '/content/drive/My Drive/TCC/'

Mounted at /content/drive/


In [4]:
n_classes = 6
labels_to_int = {"scab": 0, "rust": 1, "powdery_mildew": 2, "healthy": 3, "frog_eye_leaf_spot": 4, "complex": 5}

def to_categorical(labels):
    """Turns labels string into categorical n-hot representation"""
    categorical = []
    for lbl in labels:
        cat = [0] * n_classes
        for key,val in labels_to_int.items():
            if key in lbl:
                cat[val] = 1
        categorical.append(cat)
    return np.array(categorical)

In [5]:
from PIL import Image
def load_images(filenames, path):
    data = []
    for fn in filenames:
        img = np.array(Image.open(path+fn))
        data.append(img)
    # Preprocessing
    data = np.array(data).astype('float32')
    data /= 255.0
    mean_pixel = np.mean(data)
    data -= mean_pixel # subtracting mean pixel vcalue (from the dataset) as suggested in VGG paper
    return data

In [6]:
# Loading the labels
labels_df = pd.read_csv(path+'train.csv')

In [10]:
# Loading the train dataset
img_path = path+"resized/"

train_img_fns = labels_df.iloc[:5000,0].values
x_train = load_images(train_img_fns, img_path)

labels_train = labels_df.iloc[:5000,1].values
y_train = to_categorical(labels_train)

In [11]:
# Loading the test dataset
test_img_fns = labels_df.iloc[5000:6000,0].values
x_test = load_images(test_img_fns, img_path)

labels_test = labels_df.iloc[5000:6000,1].values
y_test = to_categorical(labels_test)

## Network

In [12]:
from tensorflow import keras
from tensorflow.keras import layers

In [13]:
input_shape = (224, 224, 3)

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        
        # Stack of convolutional layers
        layers.Conv2D(64, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.Conv2D(64, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        
        layers.Conv2D(128, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.Conv2D(128, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        
        layers.Conv2D(256, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.Conv2D(256, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.Conv2D(256, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        
        layers.Conv2D(512, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.Conv2D(512, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.Conv2D(512, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        
        layers.Conv2D(512, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.Conv2D(512, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.Conv2D(512, kernel_size=(3, 3), padding="same", activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        
        # Fully-Connected layers
        layers.Flatten(),
        layers.Dense(4096, activation='relu'),
        layers.Dense(4096, activation='relu'),
        layers.Dense(n_classes, activation='sigmoid'),
        
    ]
)

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 256)       2

In [15]:
# Train the model
batch_size = 50
epochs = 15

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/15
100/100 [==============================] - 153s 1s/step - loss: 1.7977 - accuracy: 0.3038
Epoch 2/15
100/100 [==============================] - 133s 1s/step - loss: 0.4491 - accuracy: 0.3054
Epoch 3/15
100/100 [==============================] - 133s 1s/step - loss: 0.4490 - accuracy: 0.3076
Epoch 4/15
100/100 [==============================] - 133s 1s/step - loss: 0.4490 - accuracy: 0.3160
Epoch 5/15
100/100 [==============================] - 133s 1s/step - loss: 0.4484 - accuracy: 0.3160
Epoch 6/15
100/100 [==============================] - 132s 1s/step - loss: 0.4480 - accuracy: 0.3160
Epoch 7/15
100/100 [==============================] - 132s 1s/step - loss: 0.4481 - accuracy: 0.3160
Epoch 8/15
100/100 [==============================] - 132s 1s/step - loss: 0.4479 - accuracy: 0.3160
Epoch 9/15
100/100 [==============================] - 132s 1s/step - loss: 0.4484 - accuracy: 0.3160
Epoch 10/15
100/100 [==============================] - 132s 1s/step - loss: 0.4483 - accura

In [16]:
# Test the model 
score = model.evaluate(x_test, y_test, verbose=0)
score

[0.43971624970436096, 0.31299999356269836]

In [17]:
loss = score[0]
accuracy = score[1]
print(f"loss = {loss}, accuracy = {accuracy}")

loss = 0.43971624970436096, accuracy = 0.31299999356269836
